# YOLOv3 from scratch with PyTorch

https://arxiv.org/abs/1506.02640

### Imports

In [1]:
import torch
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from model import YOLO, yolov3_config
from dataset import YOLOv3Dataset
from loss import YOLOv3Loss
from utils import (
    load_checkpoint,
    trainable_parameters,
)
from train import train_yolov3
from test import test_model

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.12 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


### Constants

In [2]:
LEARNING_RATE = 2e-5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
WEIGHT_DECAY = 0
EPOCHS = 100
NUM_WORKERS = 6
PIN_MEMORY = True
LOAD_MODEL = False
LOAD_MODEL_FILE = "yolov3-20.pt"
LOAD_MODEL_EPOCH = 0
# https://www.kaggle.com/datasets/aladdinpersson/pascalvoc-yolo
IMG_DIR = "/data/Datasets/pascalvoc/images"
LABEL_DIR = "/data/Datasets/pascalvoc/labels"
TRAIN_CSV = "/data/Datasets/pascalvoc/train.csv"
TEST_CSV = "/data/Datasets/pascalvoc/test.csv"
IMAGE_SIZE = 416
S = [IMAGE_SIZE // 32, IMAGE_SIZE // 16, IMAGE_SIZE // 8]
NUM_CLASSES = 20
ANCHORS = [
    [(0.28, 0.22), (0.38, 0.48), (0.9, 0.78)],
    [(0.07, 0.15), (0.15, 0.11), (0.14, 0.29)],
    [(0.02, 0.03), (0.04, 0.07), (0.08, 0.06)],
]

### Initialization of the model

In [3]:
model = YOLO(yolov3_config).to(DEVICE)
params = trainable_parameters(model)
print(f"Trainable parameters: {params}")
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
loss_fn = YOLOv3Loss()

if LOAD_MODEL:
    load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)

Layer 0: Conv with 3 input channels and 32 output channels
Layer 1: Conv with 32 input channels and 64 output channels
Layer 2: Bottleneck with 64 input channels and 64 output channels
Layer 3: Conv with 64 input channels and 128 output channels
Layer 4: Bottleneck with 128 input channels and 128 output channels
Layer 5: Bottleneck with 128 input channels and 128 output channels
Layer 6: Conv with 128 input channels and 256 output channels
Layer 7: Bottleneck with 256 input channels and 256 output channels
Layer 8: Bottleneck with 256 input channels and 256 output channels
Layer 9: Bottleneck with 256 input channels and 256 output channels
Layer 10: Bottleneck with 256 input channels and 256 output channels
Layer 11: Bottleneck with 256 input channels and 256 output channels
Layer 12: Bottleneck with 256 input channels and 256 output channels
Layer 13: Bottleneck with 256 input channels and 256 output channels
Layer 14: Bottleneck with 256 input channels and 256 output channels
Layer 1

### Initialization of the training dataset/dataloader

In [4]:
train_transforms = A.Compose(
    [
        A.LongestMaxSize(max_size=int(IMAGE_SIZE * 1.1)),
        A.PadIfNeeded(
            min_height=int(IMAGE_SIZE * 1.1),
            min_width=int(IMAGE_SIZE * 1.1)
        ),
        A.RandomCrop(width=IMAGE_SIZE, height=IMAGE_SIZE),
        A.ColorJitter(brightness=0.6, contrast=0.6, saturation=0.6, hue=0.6, p=0.4),
        A.ShiftScaleRotate(
            rotate_limit=20, p=0.5, border_mode=cv2.BORDER_CONSTANT
        ),
        A.HorizontalFlip(p=0.5),
        A.Blur(p=0.1),
        A.CLAHE(p=0.1),
        A.Posterize(p=0.1),
        A.ToGray(p=0.1),
        A.ChannelShuffle(p=0.05),
        A.Normalize(mean=[0, 0, 0], std=[1, 1, 1], max_pixel_value=255,),
        ToTensorV2(),
    ],
    bbox_params=A.BboxParams(format="yolo", min_visibility=0.4, label_fields=[],),
)

test_transforms = A.Compose(
    [
        A.LongestMaxSize(max_size=IMAGE_SIZE),
        A.PadIfNeeded(min_height=IMAGE_SIZE, min_width=IMAGE_SIZE),
        A.Normalize(mean=[0, 0, 0], std=[1, 1, 1], max_pixel_value=255,),
        ToTensorV2(),
    ],
    bbox_params=A.BboxParams(format="yolo", min_visibility=0.4, label_fields=[]),
)

train_dataset = YOLOv3Dataset(
    "/data/Datasets/pascalvoc/100examples.csv",
    img_dir=IMG_DIR,
    label_dir=LABEL_DIR,
    anchors=ANCHORS,
    split_sizes=S,
    num_classes=NUM_CLASSES,
    #transform=train_transforms,
    transform=test_transforms,
)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=True,
    drop_last=True,
)

test_dataset = YOLOv3Dataset(
    "/data/Datasets/pascalvoc/100examples.csv",
    img_dir=IMG_DIR,
    label_dir=LABEL_DIR,
    anchors=ANCHORS,
    split_sizes=S,
    num_classes=NUM_CLASSES,
    transform=test_transforms,
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=True,
    drop_last=True,
)

/data/Projets/IA/yolov1/venv/lib/python3.12/site-packages/pydantic/main.py:363: UserWarning: Pydantic serializer warnings:
  Expected `Union[float, tuple[float, float]]` but got `list` - serialized value may not be as expected
  Expected `Union[float, tuple[float, float]]` but got `list` - serialized value may not be as expected
  Expected `Union[float, tuple[float, float]]` but got `list` - serialized value may not be as expected
  Expected `Union[float, tuple[float, float]]` but got `list` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


### Model training

In [5]:
train_yolov3(model, train_loader, test_loader, optimizer, loss_fn, DEVICE, EPOCHS, S, ANCHORS)

100%|██████████| 6/6 [00:07<00:00,  1.30s/it, loss=47.8, mean_loss=53.2]


Epoch [1/100], Mean loss: 53.1945


100%|██████████| 6/6 [00:05<00:00,  1.10it/s, loss=51.3, mean_loss=52.8]


Epoch [2/100], Mean loss: 52.8289


100%|██████████| 6/6 [00:05<00:00,  1.15it/s, loss=45.7, mean_loss=47.2]


...

100%|██████████| 6/6 [00:08<00:00,  1.49s/it, loss=25.8, mean_loss=25.2]


Epoch [69/100], Mean loss: 25.2416


100%|██████████| 6/6 [00:08<00:00,  1.47s/it, loss=25.8, mean_loss=25.3]


Epoch [70/100], Mean loss: 25.3224


 83%|████████▎ | 5/6 [01:08<00:13, 13.93s/it]

### Initialization of the testing dataset/dataloader

In [ ]:
test_dataset = YOLOv3Dataset(
    "/data/Datasets/pascalvoc/test.csv",
    img_dir=IMG_DIR,
    label_dir=LABEL_DIR,
    anchors=ANCHORS,
    split_sizes=S,
    num_classes=NUM_CLASSES,
    transform=test_transforms,
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=True,
    drop_last=True,
)

### Plot some predictions on the test dataset

In [ ]:
test_model(10, model, test_loader, DEVICE, split_size=SPLIT_SIZE, num_boxes=NUM_BOXES, num_classes=NUM_CLASSES)

NameError: name 'SPLIT_SIZE' is not defined